## Metrics Computation
Compute raw scores using BRISQUE, NIQE, and PIQE via PyIQA (or similar libraries).

In [26]:
import pyiqa

image = "koniq10k_1024x768/139611767.jpg"

brisque_metric = pyiqa.create_metric("brisque")
niqe_metric = pyiqa.create_metric("niqe")
piqe_metric = pyiqa.create_metric("piqe")

brisque_score = brisque_metric(image)
print(brisque_score.item())

niqe_score = niqe_metric(image)
print(niqe_score.item())

piqe_score = piqe_metric(image)
print(piqe_score.item())

19.88836669921875
5.656121928129492
17.730968475341797


## Refined Out-of-Bounds Check

BRISQUE: Theoretically ranges 0–100, where lower is better. Escalate if the score < 0 or > 100 .

NIQE: Starts at 0 (best), no strict upper bound. In practice, values for most natural images lie around 3–20. Escalate if score < 0 or > 25.

PIQE: Theoretical range is 0–100. Escalate if outside that range (rare in practice).

Escalate directly to heavy IQA (MANIQA/HyperIQA) if any score is out-of-bounds.

In [27]:
if brisque_score.item() < 0 or brisque_score.item() > 100:
    print("BRISQUE score out of bounds, escalate to deep IQA.")

if niqe_score.item() < 0 or niqe_score.item() > 100:
    print("NIQE score out of bounds, escalate to deep IQA.")

if piqe_score.item() < 0 or piqe_score.item() > 100:
    print("PIQE score out of bounds, escalate to deep IQA.")

## Normalization of Metrics
If all scores are valid:

Normalize into [0, 1]. For example:

brisque_norm = (brisque - 0) / 100

niqe_norm    = (niqe - 0) / 25

piqe_norm    = (piqe - 0) / 100

Direction remains: lower normalized value = better quality.

In [28]:
brisque_norm = brisque_score.item() / 100
niqe_norm = niqe_score.item() / 25
piqe_norm = piqe_score.item() / 100

normalized_scores = [brisque_norm, niqe_norm, piqe_norm]

print("Normalized BRISQUE:", brisque_norm)
print("Normalized NIQE:", niqe_norm)
print("Normalized PIQE:", piqe_norm)

Normalized BRISQUE: 0.1988836669921875
Normalized NIQE: 0.22624487712517968
Normalized PIQE: 0.17730968475341796


## Consistency Check

Compute a disagreement metric among normalized scores, such as:

Variance (e.g. np.var([brisque_norm, niqe_norm, piqe_norm])), or

Range (max − min).

If disagreement ≤ threshold (e.g., 0.1), output the average normalized score.

If > threshold, escalate to a deep IQA model.

In [29]:
import numpy as np


variance = np.var(normalized_scores)
score_range = max(normalized_scores) - min(normalized_scores)
threshold = 0.1

print("Variance:", variance)
print("Range:", score_range)

if variance <= threshold and score_range <= threshold:
    avg_score = np.mean(normalized_scores)
    print("Average normalized score:", avg_score)
else:
    print("High disagreement detected, escalate to deep IQA model.")

Variance: 0.0004009695091157582
Range: 0.04893519237176172
Average normalized score: 0.20081274295692841


## Deep Model Fallback

Use a more reliable deep IQA method (MANIQA, HyperIQA, etc.) to get a final MOS-like score when:

Any metric is out-of-bounds, or

There is high disagreement between the traditional metrics.